In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import cv2
import os
import time
from IPython.display import clear_output
from sklearn.utils import shuffle
from sklearn import metrics as sk_metrics
from sklearn.model_selection import RandomizedSearchCV
import pickle as pkl

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
import xgboost

from dataset import *
from descriptor import *
from SlidingWindow import *
from BagOfWords import *

In [ ]:
from classification import *
surf_classifier = PreparedClassifier('waldo', 'simple')
sift_classifier_fir = PreparedClassifier('waldo', 'hard')
sift_classifier_sec = PreparedClassifier('waldo', 'mix')

In [7]:
from detection import *
cascade_classifier = load_haar_classifier()

In [11]:
img_bboxes = []
img_path = os.path.join(os.getcwd(), 'datasets', 'JPEGImages', '074.jpg')
image = cv2.imread(img_path)
detections = []

In [12]:
haar_boxes = cascade_classifier.detectMultiScale(image, scaleFactor=1.05, minNeighbors=5,
                                                             minSize=(100, 100), maxSize=(400, 400))

In [14]:
for box in haar_boxes:
    x = box[0]
    y = box[1]
    x_end = x + box[2]
    y_end = y + box[3]
    window = image[y:y_end, x:x_end]

    predict_score = surf_classifier.predict_proba([window])[0][1]
    if predict_score >= 0.7:
        detections.append((x, y, x_end, y_end, predict_score))

AttributeError: 'PreparedClassifier' object has no attribute 'metrics'